In [56]:
import numpy as np
import torch
from torch.autograd.functional import hessian
from torch.nn.functional import sigmoid

n = 10


def autograd_df(f, x):
    y = f(x)
    y.backward()
    return x.grad

def autograd_ddf(f, x):
    return hessian(f, x, strict=True)

## a.

$$f(x) = \dfrac{1}{2}x^TAx + b^Tx + c $$

In [78]:
A = torch.rand((n,n))
b = torch.rand(n)
c = torch.rand(1)


def f(x):
    return 0.5 * x.t() @ A @ x + b.t() @ x + c

def analytical_df(x):
    return 0.5 * (A + A.t()) @ x + b

def analytical_ddf(x):
    return 0.5 * (A + A.t())



x_test = torch.rand(n, requires_grad=True)

diff = np.allclose(analytical_df(x_test.detach()), autograd_df(f, x_test).detach().numpy())
print(f'Analytical and autograd implementations of the gradients are close: {diff}')
diff = np.allclose(analytical_ddf(x_test.detach()), autograd_ddf(f, x_test).detach())
print(f'Analytical and autograd implementations of the hessians are close: {diff}')

Analytical and autograd implementations of the gradients are close: True
Analytical and autograd implementations of the hessians are close: True


## b.

$$f(x) = \ln \left( 1 + \exp\langle a,x\rangle\right)$$ 

In [147]:
a = torch.rand(n)

def f(x):
    return torch.log(1 + torch.exp(a.dot(x)))

def analytical_df(x):
    return sigmoid(a.dot(x)) * a

def analytical_ddf(x):
    return sigmoid(a.dot(x)) * (1 - sigmoid(a.dot(x))) * (a.unsqueeze(1) @ a.unsqueeze(1).t())



x_test = torch.rand(n, requires_grad=True)

diff = np.allclose(analytical_df(x_test.detach()), autograd_df(f, x_test).detach().numpy())
print(f'Analytical and autograd implementations of the gradients are close: {diff}')
diff = np.allclose(analytical_ddf(x_test.detach()), autograd_ddf(f, x_test).detach())
print(f'Analytical and autograd implementations of the hessians are close: {diff}')

Analytical and autograd implementations of the gradients are close: True
Analytical and autograd implementations of the hessians are close: True


## c.

$$f(x) = \dfrac{1}{2} \|Ax - b\|^2_2$$

In [175]:
A = torch.rand((n,n))
b = torch.rand(n)

def f(x):
    return 0.5 * torch.norm(A @ x - b, 2)**2

def analytical_df(x):
    return A.t() @ (A @ x - b)

def analytical_ddf(x):
    return A.t() @ A



x_test = torch.rand(n, requires_grad=True)

diff = np.allclose(analytical_df(x_test.detach()), autograd_df(f, x_test).detach().numpy())
print(f'Analytical and autograd implementations of the gradients are close: {diff}')
diff = np.allclose(analytical_ddf(x_test.detach()), autograd_ddf(f, x_test).detach())
print(f'Analytical and autograd implementations of the hessians are close: {diff}')

Analytical and autograd implementations of the gradients are close: True
Analytical and autograd implementations of the hessians are close: True
